In [2]:
import tensorflow as tf
from tensorflow.keras import layers

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Load Data



In [3]:
# Load data
data = pd.read_csv('https://raw.githubusercontent.com/tummyfit-app/TummyFit-ML/main/Datasets/data-tummyfit-v4.csv')
data

,Recipe ID,Recipe Title,Calories,Fat/g,Carbo/g,Protein/g,Alcohol,Vegetarian,Vegan,Gluten Free,...,Ready in Minutes,Image,Popular,Price Per Serving,Halal,Breakfast,Lunch,Dinner,Snack 1,Snack 2
0,1,Fried Anchovies with Sage,384.0,11.0,33.0,37.0,0.0,0,0,0,...,45,https://spoonacular.com/recipeImages/1-556x370...,0,5.6051,1,1,0,0,0,0
1,2,Anchovies Appetizer With Breadcrumbs & Scallions,57.0,2.0,4.0,5.0,0.0,0,0,0,...,15,https://spoonacular.com/recipeImages/2-556x370...,0,8.2060,1,0,0,0,1,0
2,3,Fried Anchovies,356.0,31.0,10.0,9.0,0.0,0,0,0,...,15,https://spoonacular.com/recipeImages/6-556x370...,0,1.5050,1,0,0,0,1,0
3,4,Marinated Fresh Anchovies: Alici Marinate,221.0,11.0,3.0,24.0,0.0,0,0,1,...,420,https://spoonacular.com/recipeImages/9-556x370...,0,5.5005,0,1,0,0,0,0
4,5,Roasted Peppers with Boquerones,32.0,0.0,5.0,2.0,0.0,0,0,1,...,45,https://spoonacular.com/recipeImages/12-556x37...,0,5.8440,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,1171,Lentil Bolognese with Zucchini Noodles,350.0,12.0,45.0,15.0,0.0,1,1,1,...,40,https://minimalistbaker.com/wp-content/uploads...,1,7.0000,1,0,0,1,0,0
1171,1172,Portobello Mushroom Burgers,300.0,10.0,40.0,15.0,0.0,1,1,1,...,20,https://dishingouthealth.com/wp-content/upload...,1,8.0000,1,0,0,1,0,0
1172,1173,Caprese Salad,200.0,12.0,10.0,10.0,0.0,1,0,1,...,10,https://natashaskitchen.com/wp-content/uploads...,1,6.0000,1,0,0,1,0,0
1173,1174,Teriyaki Salmon,400.0,20.0,20.0,35.0,0.0,0,0,1,...,25,https://www.dinneratthezoo.com/wp-content/uplo...,1,12.0000,1,0,0,1,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1175 entries, 0 to 1174
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recipe ID          1175 non-null   int64  
 1   Recipe Title       1175 non-null   object 
 2   Calories           1175 non-null   float64
 3   Fat/g              1175 non-null   float64
 4   Carbo/g            1175 non-null   float64
 5   Protein/g          1175 non-null   float64
 6   Alcohol            1175 non-null   float64
 7   Vegetarian         1175 non-null   int64  
 8   Vegan              1175 non-null   int64  
 9   Gluten Free        1175 non-null   int64  
 10  Dairy Free         1175 non-null   int64  
 11  Dish Type          1175 non-null   object 
 12  Ingredients        1175 non-null   object 
 13  Instructions       1174 non-null   object 
 14  Ready in Minutes   1175 non-null   int64  
 15  Image              1175 non-null   object 
 16  Popular            1175 

# Model

In [5]:
# Load food_data as y
food_data = data

# Preprocess input data
def preprocess_input(calorie_requirement, halal, vegetarian, vegan, gluten_free, dairy_free):
    # Normalize calorie_requirement to range [0, 1]
    calorie_requirement /= 3000.0

    input_data = [calorie_requirement, halal, vegetarian, vegan, gluten_free, dairy_free]
    return np.array(input_data)

In [6]:
# Generate training data
# Buat dummy dataset untuk X train (train_input), berisi 5000 data user input
# dan y train (train_output), berisi kombinasi menu harian
train_input = []
train_output = []

for _ in range(5000):
    # Generate random input values
    calorie_requirement = np.random.randint(500, 3001)
    halal = bool(np.random.randint(2))
    vegetarian = bool(np.random.randint(2))
    vegan = bool(np.random.randint(2))
    gluten_free = bool(np.random.randint(2))
    dairy_free = bool(np.random.randint(2))

    # Preprocess input
    input_data = preprocess_input(calorie_requirement, halal, vegetarian, vegan, gluten_free, dairy_free)

    # Generate random menu combination
    menu_combination = []

    # Select one menu for each category
    categories = ['Breakfast', 'Lunch', 'Dinner', 'Snack 1', 'Snack 2']
    for category in categories:
        category_menu = food_data[food_data[category] == 1].reset_index(drop=True)
        menu_index = np.random.choice(len(category_menu))
        menu_combination.append(menu_index)

    # Append to training data
    train_input.append(input_data)
    train_output.append(menu_combination)

train_input = np.array(train_input)
train_output = np.array(train_output)

In [7]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(6,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5)
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Train model
model.fit(train_input, train_output, epochs=100, batch_size=64)

Epoch 1/100
79/79 [==============================] - 1s 4ms/step - loss: 10494.9658
Epoch 2/100
79/79 [==============================] - 0s 4ms/step - loss: 5227.2959
Epoch 3/100
79/79 [==============================] - 0s 3ms/step - loss: 5099.5942
Epoch 4/100
79/79 [==============================] - 0s 4ms/step - loss: 5064.8506
Epoch 5/100
79/79 [==============================] - 0s 4ms/step - loss: 5057.8315
Epoch 6/100
79/79 [==============================] - 0s 4ms/step - loss: 5054.1089
Epoch 7/100
79/79 [==============================] - 0s 4ms/step - loss: 5053.0898
Epoch 8/100
79/79 [==============================] - 0s 3ms/step - loss: 5066.0464
Epoch 9/100
79/79 [==============================] - 0s 3ms/step - loss: 5063.1455
Epoch 10/100
79/79 [==============================] - 0s 4ms/step - loss: 5052.7812
Epoch 11/100
79/79 [==============================] - 0s 4ms/step - loss: 5047.5986
Epoch 12/100
79/79 [==============================] - 0s 3ms/step - loss: 5052.9829


# Save and Load Model

In [8]:
# Save model
model_json = model.to_json()
with open("tummyfit_model.json", "w") as json_file:
    json_file.write(model_json)

# Save weights
model.save_weights("tummyfit_model_weights.h5")

In [9]:
# Load model & weights
with open("tummyfit_model.json", "r") as json_file:
    loaded_model_json = json_file.read()

model = tf.keras.models.model_from_json(loaded_model_json)
model.load_weights("tummyfit_model_weights.h5")

# Compile loaded model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Menghitung Kebutuhan Kalori Harian

In [10]:
def calculate_daily_calorie_requirement(weight, height, sex, age, daily_activity, goal):
    # Menghitung BMR (Basal Metabolic Rate)
    if sex == "male":
        bmr = 66 + (13.75 * weight) + (5 * height) - (6.75 * age)
    elif sex == "female":
        bmr = 655 + (9.56 * weight) + (1.85 * height) - (4.68 * age)
    else:
        raise ValueError("Invalid sex. Please enter 'male' or 'female'.")

    # Menyesuaikan kebutuhan kalori berdasarkan aktivitas harian
    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725,
        "extra active": 1.9
    }
    if daily_activity not in activity_factors:
        raise ValueError("Invalid daily activity. Please choose from: 'sedentary', 'lightly active', 'moderately active', 'very active', 'extra active'.")
    
    daily_calorie_requirement = bmr * activity_factors[daily_activity]

    # Menyesuaikan kebutuhan kalori berdasarkan tujuan
    if goal == "Maintain weight":
        return daily_calorie_requirement
    elif goal == "Weight loss":
        # Mengurangi 500 kalori per hari untuk tujuan penurunan berat badan
        calorie_deficit = 500
        return daily_calorie_requirement - calorie_deficit
    else:
        raise ValueError("Invalid goal. Please choose from: 'Maintain weight' or 'Weight loss'.")

In [11]:
# Test perhitungan kalori dari data from database

weight = 70  # Berat dalam kilogram
height = 170  # Tinggi dalam sentimeter
sex = "male"  # Jenis kelamin: "male" atau "female"
age = 30  # Usia dalam tahun
daily_activity = "moderately active"  # Aktivitas harian
goal = "Weight loss" # Tujuan menghitung kebutuhan kalori

calorie_requirement = calculate_daily_calorie_requirement(weight, height, sex, age, daily_activity, goal)
print(f"Kebutuhan kalori per hari: {calorie_requirement} kalori")

Kebutuhan kalori per hari: 2097.8 kalori


#User Preferences

In [12]:
halal = True
vegetarian = False
vegan = False
gluten_free = False
dairy_free = False

# Test Model

In [13]:
# Generate menu based on user input
def generate_menu(model, input_data):
    input_data = np.array([input_data])
    menu_indices = model.predict(input_data)[0].astype(int)

    menu_combination = []
    categories = ['Breakfast', 'Lunch', 'Dinner', 'Snack 1', 'Snack 2']
    for index, category in enumerate(categories):
        category_menu = food_data[food_data[category] == 1]

        # Filter menu based on calorie requirement and preferences
        filtered_menu = category_menu[
            (category_menu['Calories'] <= (calorie_requirement - sum(menu['Calories'] for menu in menu_combination))) &
            (category_menu['Halal'] == int(halal)) &
            (category_menu['Vegetarian'] == int(vegetarian)) &
            (category_menu['Vegan'] == int(vegan)) &
            (category_menu['Gluten Free'] == int(gluten_free)) &
            (category_menu['Dairy Free'] == int(dairy_free))
        ]

        if len(filtered_menu) > 0:
            menu_index = np.random.choice(len(filtered_menu))
            menu_combination.append(filtered_menu.iloc[menu_index])
        else:
            # If no menu available within the remaining calorie limit and preferences, regenerate the combination
            return generate_menu(model, input_data)

    return menu_combination


# Generate weekly meal plan
def generate_weekly_menu(model, input_data):
    weekly_menu = []
    for _ in range(7):
        menu_combination = generate_menu(model, input_data)
        weekly_menu.append(menu_combination)
    return weekly_menu

In [14]:
input_data = preprocess_input(calorie_requirement, halal, vegetarian, vegan, gluten_free, dairy_free)
weekly_menu = generate_weekly_menu(model, input_data)

1/1 [==============================] - 0s 15ms/step


In [15]:
# Print weekly menu with total calories
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print(f"Your daily calorie requirement: {calorie_requirement}")
for day, menu_combination in zip(days, weekly_menu):
    print(f"Menu for {day}:")
    print("-----------------")
    total_calories = sum(menu['Calories'] for menu in menu_combination)
    for index, category in enumerate(categories):
        menu = menu_combination[index]
        recipe_title = menu['Recipe Title']
        calories = menu['Calories']
        halal = "Yes" if menu['Halal'] == 1 else "No"
        vegetarian = "Yes" if menu['Vegetarian'] == 1 else "No"
        vegan = "Yes" if menu['Vegan'] == 1 else "No"
        gluten_free = "Yes" if menu['Gluten Free'] == 1 else "No"
        dairy_free = "Yes" if menu['Dairy Free'] == 1 else "No"
        ingredients = menu['Ingredients']
        instructions = menu['Instructions']

        print(f"{category}:")
        print("Recipe Title:", recipe_title)
        print("Calories:", calories)
        print("Halal:", halal)
        print("Vegetarian:", vegetarian)
        print("Vegan:", vegan)
        print("Gluten Free:", gluten_free)
        print("Dairy Free:", dairy_free)
        print("Ingredients:", ingredients)
        print("Instructions:", instructions)
        print("-----------------")
    print("Total Calories:", total_calories)
    print("________________________________")

Your daily calorie requirement: 2097.8
Menu for Monday:
-----------------
Breakfast:
Recipe Title: Wheat Noodles with Anchovy
Calories: 703.0
Halal: Yes
Vegetarian: No
Vegan: No
Gluten Free: No
Dairy Free: No
Ingredients: eggs,   milk,   olive oil,   onion,   parsley,   salted anchovies,   whole wheat flour
Instructions: Bring 6 quarts water to boil and add 2 tablespoons salt. Place the noodles in boiling water, and allow 8 to 9 minutes to cook. In a 12 to 14-inch saute pan, place the oil, anchovies, and onion over medium heat and cook. Stir often to break up the anchovies until a paste is formed, about 8 to 10 minutes. Drain the pasta and toss into the pan. Add the parsley, toss over high heat 1 minute and place on a plate. Serve immediately. To make the dough: Make a mound of the flour in the center of a large wooden cutting board. Make a well in the middle of the flour and add the eggs and milk mixture. Using a fork, beat together the eggs and milk mixture and begin to incorporate t